# Data exploration and prompt engineering

In [ ]:
# First upgrade pip
%pip install --upgrade pip

# Install tensorflow and keras first
%pip install tensorflow==2.18.0 keras==3.9.0

# Install torch and torchdata
%pip install --no-deps torch==2.5.1 torchdata==0.6.0 --quiet

# Then install other packages except TRL
%pip install -U \
    datasets==2.17.0 \
    transformers==4.38.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Load the datasets, Large Language Model (LLM), tokenizer, and configurator. Do not worry if you do not understand yet all of those components - they will be described and discussed later in the notebook.

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

<a name='2'></a>
## 2 - Summarize Dialogue without Prompt Engineering

In this use case, you will be generating a summary of a dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging Face. The list of available models in the Hugging Face `transformers` package can be found [here](https://huggingface.co/docs/transformers/index). 

Let's upload some simple dialogues from the [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face dataset. This dataset contains 10,000+ dialogues with the corresponding manually labeled summaries and topics. 

In [31]:
# Load the dataset

dataset = load_dataset("EdinburghNLP/xsum")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [32]:
# Use a small sample for training
train_data = dataset["train"].select(range(5000))  # or smaller if needed
val_data = dataset["validation"].select(range(1000))
test_data = dataset["test"].select(range(500))

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}, Test samples: {len(test_data)}")

Train samples: 5000, Val samples: 1000, Test samples: 500


Print a couple of dialogues with their baseline summaries.

In [33]:
example_indices = [5, 20, 25]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT ARTICLE:')
    print(dataset['train'][index]['document'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY/HIGHLIGHTS:')
    print(dataset['train'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT ARTICLE:
Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau.
Rynard Landman and Ashton Hewitt got a try in either half for the Dragons.
Glasgow showed far superior strength in depth as they took control of a messy match in the second period.
Home coach Gregor Townsend gave a debut to powerhouse Fijian-born Wallaby wing Taqele Naiyaravoro, and centre Alex Dunbar returned from long-term injury, while the Dragons gave first starts of the season to wing Aled Brew and hooker Elliot Dee.
Glasgow lost hooker Pat McArthur to an early shoulder injury but took advantage of their first pressure when Rory Clegg slotted over a penalty on 12 minutes.
It took 24 minutes for a disjointed game

Load the [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5), creating an instance of the `AutoModelForSeq2SeqLM` class with the `.from_pretrained()` method. 

In [34]:
model_name='google/flan-t5-small'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


To perform encoding and decoding, you need to work with text in a tokenized form. **Tokenization** is the process of splitting texts into smaller units that can be processed by the LLM models. 

Download the tokenizer for the FLAN-T5 model using `AutoTokenizer.from_pretrained()` method. Parameter `use_fast` switches on fast tokenizer. At this stage, there is no need to go into the details of that, but you can find the tokenizer parameters in the [documentation](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoTokenizer).

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Test the tokenizer encoding and decoding a simple sentence:

In [38]:
for i, index in enumerate(example_indices):
    document = dataset['train'][index]['document']
    summary = dataset['train'][index]['summary']
    
    inputs = tokenizer(document, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY/HIGHLIGHTS:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
(CNN)Just as mimeograph machines and photocopiers were in their day, online activity -- blogs, YouTube channels, even social media platforms like Facebook and Twitter -- have fully emerged as the alternative to traditional mainstream media. It is not just the low cost of posting online that attracts dissidence, though that in itself is liberating. It is the lack of access to traditional print and broadcast media in authoritarian countries that is really the driving force leading disaffected voices to post online. It is not unique to Asia, but it might seem more pronounced if you live there. Going online has become the path of least resistance if you want to make yourself heard. But it still brings resistance, some of it legal, some of it deadly. Let's look at the l

You can see that the guesses of the model make some sense, but it doesn't seem to be sure what task it is supposed to accomplish. Seems it just makes up the next sentence in the dialogue. Prompt engineering can help here.

<a name='3'></a>
## 3 - Summarize Dialogue with an Instruction Prompt


<a name='3.1'></a>
### 3.1 - Zero Shot Inference with an Instruction Prompt


In [42]:
for i, index in enumerate(example_indices):
    document = dataset['train'][index]['document']
    summary = dataset['train'][index]['summary']

    prompt = f"""
Summarise the following document.
{document}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=100,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)    
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarise the following document.
Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau.
Rynard Landman and Ashton Hewitt got a try in either half for the Dragons.
Glasgow showed far superior strength in depth as they took control of a messy match in the second period.
Home coach Gregor Townsend gave a debut to powerhouse Fijian-born Wallaby wing Taqele Naiyaravoro, and centre Alex Dunbar returned from long-term injury, while the Dragons gave first starts of the season to wing Aled Brew and hooker Elliot Dee.
Glasgow lost hooker Pat McArthur to an early shoulder injury but took advantage of their first pressure when Rory Clegg slotted over a penalty on 12 minutes.
It too

This is much better! But the model still does not pick up on the nuance of the conversations though.

<a name='3.2'></a>
### 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5


In [43]:
for i, index in enumerate(example_indices):
    document = dataset['train'][index]['document']
    summary = dataset['train'][index]['summary']
        
    prompt = f"""
Document:

{document}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=100,
        )[0], 
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Document:

Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau.
Rynard Landman and Ashton Hewitt got a try in either half for the Dragons.
Glasgow showed far superior strength in depth as they took control of a messy match in the second period.
Home coach Gregor Townsend gave a debut to powerhouse Fijian-born Wallaby wing Taqele Naiyaravoro, and centre Alex Dunbar returned from long-term injury, while the Dragons gave first starts of the season to wing Aled Brew and hooker Elliot Dee.
Glasgow lost hooker Pat McArthur to an early shoulder injury but took advantage of their first pressure when Rory Clegg slotted over a penalty on 12 minutes.
It took 24 minutes for a disj

Notice that this prompt from FLAN-T5 did help a bit, but still struggles to pick up on the nuance of the conversation. This is what you will try to solve with the few shot inferencing.

<a name='4'></a>
## 4 - Summarize Dialogue with One Shot and Few Shot Inference


<a name='4.1'></a>
### 4.1 - One Shot Inference

In [44]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        document = dataset['train'][index]['document']
        summary = dataset['train'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Document:

{document}

What was going on?
{summary}


"""
    
    document = dataset['train'][example_index_to_summarize]['document']
    
    prompt += f"""
Document:

{document}

What was going on?
"""
        
    return prompt

Construct the prompt to perform one shot inference:

In [45]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Document:

He is approaching the end of his 10th year in charge and thinks it is the right time to seek a fresh challenge.
Cricket Scotland chairman Keith Oliver said: "There is no doubt that the governing body of cricket in Scotland is unrecognisable from where we were in 2004.
"And the credit for this must go to Roddy and his staff."
During Smith's time as chief executive, his management team have increased from eight to 25 and turnover has quadrupled.
I am delighted that I leave an organisation in good health with a growing game and after a year of exceptional on-field performances by national teams at all levels
Cricket Scotland reported a rise in participation figures for players, coaches and umpires during those 10 years.
And the national side have secured a place at next year's World Cup finals in Australia and New Zealand by beating Kenya in a qualifying event.
Oliver, who has worked with Smith during that whole period, said: "Back then, we could not have imagined we would hav

Now pass this prompt to perform the one shot inference:

In [47]:
summary = dataset['train'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Plans for "English votes for English laws" are an "act of constitutional vandalism", Ed Miliband has warned.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The government has defended the government's plans to limit voting powers of MPs on Commons laws, saying it would "rip up" hundreds of years of constitutional practice in a standing order vote.


<a name='4.2'></a>
### 4.2 - Few Shot Inference

Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.

In [48]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Document:

He is approaching the end of his 10th year in charge and thinks it is the right time to seek a fresh challenge.
Cricket Scotland chairman Keith Oliver said: "There is no doubt that the governing body of cricket in Scotland is unrecognisable from where we were in 2004.
"And the credit for this must go to Roddy and his staff."
During Smith's time as chief executive, his management team have increased from eight to 25 and turnover has quadrupled.
I am delighted that I leave an organisation in good health with a growing game and after a year of exceptional on-field performances by national teams at all levels
Cricket Scotland reported a rise in participation figures for players, coaches and umpires during those 10 years.
And the national side have secured a place at next year's World Cup finals in Australia and New Zealand by beating Kenya in a qualifying event.
Oliver, who has worked with Smith during that whole period, said: "Back then, we could not have imagined we would hav

Now pass this prompt to perform a few shot inference:

In [49]:
summary = dataset['train'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Plans for "English votes for English laws" are an "act of constitutional vandalism", Ed Miliband has warned.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
The government has said it will limit the voting powers of MPs representing constituencies in England to a "quasi-English Parliament" if they have a "veto" on certain legislation.


<a name='5'></a>
## 5 - Generative Configuration Parameters for Inference

In [57]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Commons elections are set at last, and the government's intention is to restrict the vote on the "simple test" of English-only law.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Plans for "English votes for English laws" are an "act of constitutional vandalism", Ed Miliband has warned.

